In [1]:
from meta_opt.utils.experiment_utils import make, process_results, bcolors, plot, get_final_cparams

import os
import matplotlib.pyplot as plt
from IPython.display import HTML
import itertools
import dill as pkl

# Load

In [2]:
# configuration
CFG = {
    # training options are None because we arent training :)
    'workload': None,
    'num_iters': None,
    'eval_every': None,
    'num_eval_iters': None,
    'batch_size': None,
    'full_batch': True,
    'reset_every': None,

    # experiment options
    'experiment_name': 'wmt_miniscule',
    'load_checkpoint': True,
    'overwrite': False,  # whether to allow us to overwrite existing checkpoints or throw errors
    'directory': os.path.abspath("../"),
}
results = make(CFG)  # save to temp var `_results` so that we dont immediately overwrite `results` in case somethings wrong
processed_results = process_results(CFG, results)
# processed_results = pkl.load(open('{}/data/{}_processed.pkl'.format(CFG['directory'], CFG['experiment_name']), 'rb'))

using cpu for jax
results will be stored at: /Users/evandigiorno/Desktop/meta-opt/data/wmt_miniscule_*.pkl
we will try to load experiment checkpoint first
	loaded checkpoint from /Users/evandigiorno/Desktop/meta-opt/data/wmt_miniscule_raw.pkl, containing ['rsqrt']
note: using full_batch means we will never eval
[[  74.14557952]
 [  74.18462524]
 [  74.20706423]
 ...
 [2779.81098566]
 [2779.82242671]
 [2779.83446844]]
[[9.541917 ]
 [9.522913 ]
 [9.551964 ]
 ...
 [3.7237043]
 [3.5879958]
 [3.42213  ]]
[[0.00000000e+00]
 [2.00001523e-06]
 [4.00003046e-06]
 ...
 [2.00003022e-04]
 [2.00002018e-04]
 [2.00001014e-04]]
[[100.65    ]
 [114.950005]
 [121.55    ]
 [126.      ]
 [131.25    ]
 [135.25    ]
 [137.90001 ]
 [138.45    ]
 [146.3     ]
 [147.1     ]
 [149.5     ]
 [151.15001 ]
 [152.3     ]
 [154.65001 ]
 [155.45    ]
 [158.35    ]
 [157.15001 ]
 [161.65001 ]
 [162.85    ]
 [163.7     ]
 [164.85    ]
 [166.      ]
 [167.2     ]
 [169.5     ]
 [171.6     ]
 [172.6     ]
 [174.55    ]
 [1

In [3]:
# from meta_opt.utils.experiment_utils import save_checkpoint
# ret = {}
# for k in results[0].keys():
#     ret[k] = [r[k][0] for r in results]
#     for i in range(len(results)):
#         ret[k][i]['args']['experiment_name'] = 'cifar_wd'
# save_checkpoint(CFG, ret)

In [4]:
# baseline_results = pkl.load(open('{}/data/{}_processed.pkl'.format(CFG['directory'], 'mnist_baselines'), 'rb'))
# r = {}
# for k in baseline_results.keys(): 
#     r[k] = baseline_results[k]
#     for _k, _v in processed_results[k].items():
#         r[k][_k] = _v
# processed_results = r

# Plot
Plot and also animate the values taken by the $\{M_h\}_{h=1}^H$ coefficients during training. Each $M_h$ multiplies a disturbance from $h$ training steps ago (i.e. 0 is most recent in this plot).

In [ ]:
# ----------------------------------------
# plot a particular set of experiments
# ----------------------------------------
keys_to_plot = {
    'sgd': 'sgd',
    'momentum': 'momentum',
    'adamw': 'adamw',
    'dadamw': 'd-adap',
    'mechadamw': 'mechanic',
    # 'rmsprop',
    'hgd': 'hgd',
    'cf': 'ours (finetuned)',
    # 'ncf': 'ours (finetuned)',
    'frozen': 'ours (frozen)',
}

plots_to_make = {
              'loss': 'Training Loss',
              'eval_acc': 'Evaluation Accuracy',
              'bleu': 'BLEU Score',
              # 'M': 'Learned Coefficients',
              # 'anim': 'Animation'
}

# ----------------------------------------
# OR just plot em via regex
# ----------------------------------------
# keys_to_plot = '.*long.*'  # specific regex
# keys_to_plot = '.*'  # anything

(fig, ax), anim = plot(results, processed_results, keys_to_plot, plots_to_make, anim_bounds=(-0.15, 0.05), smoothing=None)
# ax[0].set_ylim(0.,1.5)
# ax[0].set_xlim(0, 4000)
# ax[1].set_ylim(0.6, 0.95)
# ax[2].set_ylim(0.95, 0.99)
# ax[3].set_ylim(0.95, 0.99)
# ax[4].set_ylim(-0.1, 40)
# ax[5].set_ylim(-0.05, 0.05)
# plt.savefig('{}/figs/{}.pdf'.format(CFG['directory'], CFG['experiment_name']))
plt.show()

if anim is not None:
    h = HTML(anim.to_html5_video())
    display(h)